In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/060ab22e-025e-4368-99fe-17c0722184e6/oauth2/token"}

dbutils.fs.mount(
source = "abfss://container@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/marketbasket",
extra_configs = configs)

True

In [ ]:
%fs

ls "mnt/marketbasket"

path,name,size,modificationTime
dbfs:/mnt/marketbasket/raw-data/,raw-data/,0,1724760230000
dbfs:/mnt/marketbasket/transformed-data/,transformed-data/,0,1724760252000


In [ ]:
basket = spark.read.format("csv").option("header","true").load("/mnt/marketbasket/raw-data/groceries.csv")

In [ ]:
basket.show()

+-------+--------------------+-------------------+--------------------+--------------------+------------------+------+------+-------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|Item(s)|              Item 1|             Item 2|              Item 3|              Item 4|            Item 5|Item 6|Item 7|       Item 8|Item 9|Item 10|Item 11|Item 12|Item 13|Item 14|Item 15|Item 16|Item 17|Item 18|Item 19|Item 20|Item 21|Item 22|Item 23|Item 24|Item 25|Item 26|Item 27|Item 28|Item 29|Item 30|Item 31|Item 32|
+-------+--------------------+-------------------+--------------------+--------------------+------------------+------+------+-------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|      

In [ ]:
basket = basket.drop(basket.columns[0])

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

# Add a unique ID to each row
basket = basket.withColumn("id", monotonically_increasing_id())

# Filter out the row where 'id' is the smallest (equivalent to dropping the first row)
first_row_id = basket.select("id").first()[0]
basket = basket.filter(basket.id != first_row_id)

# Drop the 'id' column as it's no longer needed
basket = basket.drop("id")


In [ ]:
basket.show()

+--------------------+----------------+--------------------+--------------------+------------------+------+------+-------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|              Item 1|          Item 2|              Item 3|              Item 4|            Item 5|Item 6|Item 7|       Item 8|Item 9|Item 10|Item 11|Item 12|Item 13|Item 14|Item 15|Item 16|Item 17|Item 18|Item 19|Item 20|Item 21|Item 22|Item 23|Item 24|Item 25|Item 26|Item 27|Item 28|Item 29|Item 30|Item 31|Item 32|
+--------------------+----------------+--------------------+--------------------+------------------+------+------+-------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|      tropical fruit|          yogurt| 

In [ ]:
basket.repartition(1).write.mode("overwrite").csv("/mnt/marketbasket/transformed-data/groceries")